In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os,re
from tqdm import tqdm
from sklearn.utils import class_weight
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.parallel import DataParallel
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
import pandas as np
import numpy as np
import matplotlib.pyplot as plt


In [4]:
#this funcations concat all file names of faces
folder_path = "branddataset/images"  
file_list = os.listdir(folder_path)
imgs=[]
for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        imgs.append(filename)

In [5]:
len(imgs)

44326

In [6]:
df=pd.read_csv("branddataset/brand_info.csv")

In [7]:
df["ID"]=df["ID"].astype(str)
df["ID"]=df["ID"]+".jpg"

In [8]:
df["ID"]

0        39386.jpg
1        59263.jpg
2        53759.jpg
3        29114.jpg
4         9204.jpg
           ...    
15132    39336.jpg
15133    46835.jpg
15134    19731.jpg
15135    50950.jpg
15136    48193.jpg
Name: ID, Length: 15137, dtype: object

In [9]:
real=df["ID"]

def find_fake(large_arr,small_arr):
    small_set=set(small_arr)
    arr=[]
    for x in large_arr:
        if x not in small_set:
            arr.append(x)
    return arr

not_real=find_fake(imgs,real)

In [10]:
len(not_real),len(real)

(29200, 15137)

In [11]:
real

0        39386.jpg
1        59263.jpg
2        53759.jpg
3        29114.jpg
4         9204.jpg
           ...    
15132    39336.jpg
15133    46835.jpg
15134    19731.jpg
15135    50950.jpg
15136    48193.jpg
Name: ID, Length: 15137, dtype: object

In [12]:
real_y=np.ones(len(real))
not_real_y=np.zeros(len(not_real))

In [13]:
real_y.shape

(15137,)

In [14]:
not_real_y.shape

(29200,)

In [24]:
import torchvision.models as models
import torch.nn as nn


In [30]:
def load_resnet18_model():
    resnet18 = models.resnet18(pretrained=True)
    num_feat=resnet18.fc.in_features
    resnet18.fc=nn.Softmax(2)
    resnet18.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "mps")
    resnet18 = resnet18.to(device)
    return resnet18,device

In [17]:
def pre_process_img(image):
    preprocess=transforms.Compose([transforms.ToTensor(),transforms.Resize((224, 224)),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    preprocessed_image = preprocess(image)
    preprocessed_image = preprocessed_image.unsqueeze(0)
    return preprocessed_image

In [50]:

def train_resnet18_model(model, device, train_loader, n_epoch, lr):
    print("loading model")
    criterion = nn.BCELoss()
    optimizer=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    for epoch in range(n_epoch):
        model.train()
        running_loss=0.9
        
        for inputs,labels in train_loader:
            inputs,labels=inputs.to(device),labels.to(device)
            optimizer.zero_grad()
            outputs=model(inputs)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
        epoch_loss=running_loss/len(train_loader)
        print(f"Epoch{epoch+1}/{num_epochs},Loss:{epoch_loss}:.4f")
    print("training finised")

In [31]:
resnet18_model,device = load_resnet18_model()
num_epochs = 10
learning_rate = 0.001


In [59]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import torchvision.transforms as transforms

# Step 1: Define a custom dataset class
class ImageDataset(Dataset):
    def __init__(self, data_dir_1,arr1, data_dir_2, arr2,transform=None):
        self.data_dir_1 = data_dir_1
        self.data_dir_2 = data_dir_2
        class_1_set=set(arr1)
        class_2_set=set(arr2)
        self.class_1_files = sorted([f for f in os.listdir(data_dir_1) if f.endswith('.jpg') and f in class_1_set])
        self.class_2_files = sorted([f for f in os.listdir(data_dir_2) if f.endswith('.jpg') and f in class_2_set])
        self.labels = [0] * len(self.class_1_files) + [1] * len(self.class_2_files)
        self.image_files = [os.path.join(data_dir_1, image_file) for image_file in self.class_1_files]+[os.path.join(data_dir_2, image_file) for image_file in self.class_2_files]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[index])

        return image, label


# Step 2: Create the custom dataset and DataLoader
# Replace 'path/to/your/data_folder/' with the actual path to your data folder
data_folder_path = "branddataset/images/"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

custom_dataset = ImageDataset(data_folder_path,not_real, data_folder_path, real,transform=transform)

# Define batch size and other DataLoader parameters
batch_size = 32
shuffle = True
num_workers = 4


train_ratio = 0.8  # 80% of the data for training, adjust as needed
train_size = int(train_ratio * len(custom_dataset))
test_size = len(custom_dataset) - train_size

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

batch_size = 32
shuffle = True
num_workers = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# # Step 5: Iterate through the DataLoaders to access batches of images and labels during training and testing
# for images, labels in train_loader:

#     pass

# for images, labels in test_loader:

#     pass









In [ ]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/basantkaur/miniforge3/envs/tensorflow2/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/basantkaur/miniforge3/envs/tensorflow2/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ImageDataset' on <module '__main__' (built-in)>
